In [ ]:
import pandas as pd

import numpy as np
np.random.seed(1337)


dataset_A = pd.read_csv('TSPA.csv', sep=';', names=['x', 'y', 'cost'])
dataset_B = pd.read_csv('TSPB.csv', sep=';', names=['x', 'y', 'cost'])

dataset_A.shape, dataset_B.shape

In [ ]:
node1, node2 = dataset_B.iloc[177], dataset_B.iloc[43]

print(f'{node1}\n\n{node2}')

In [ ]:
def euclidean_distance(node1, node2):
    return np.int32(((node1['x'] - node2['x']) ** 2 + (node1['y'] - node2['y']) ** 2) ** 0.5 + 0.5)

In [ ]:
print(f'distance between node1 and node2 = {euclidean_distance(node1, node2)}')

In [ ]:
def nodes_cost(node1, node2):
    return node1['cost'] + node2['cost']

In [ ]:
print(f'objective function of node1 and node2 = {euclidean_distance(node1, node2) + nodes_cost(node1, node2)}')

In [ ]:
def calculate_function_cost(dataset):
    cost = 0

    x = dataset['x'].to_numpy()
    y = dataset['y'].to_numpy()

    for i in range(len(x) - 1):
        cost += euclidean_distance({'x': x[i], 'y': y[i]}, {'x': x[i+1], 'y': y[i+1]})

    cost += euclidean_distance({'x': x[-1], 'y': y[-1]}, {'x': x[0], 'y': y[0]})

    cost += dataset['cost'].sum()

    return int(cost)

In [ ]:
def calculate_distance_matrix(dataset):
    num_nodes = len(dataset)

    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float64)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                node1 = dataset.iloc[i]
                node2 = dataset.iloc[j]
                distance = euclidean_distance(node1, node2)

                if np.isinf(distance) or np.isnan(distance):
                    raise ValueError(f"Invalid distance encountered between nodes {i} and {j}")

                cost = nodes_cost(node1, node2)
                distance_matrix[i, j] = distance + cost

    return distance_matrix

In [ ]:
distance_matrices = []

for dataset in [dataset_A, dataset_B]:
    distance_matrices.append(calculate_distance_matrix(dataset))

In [ ]:
print(f'objective function of node1 and node2 = {distance_matrices[1][177, 43]}')

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

def plot(dataset, solution=None):
    max_x = dataset['x'].max()
    max_y = dataset['y'].max()

    aspect_ratio = int(max_x / max_y)

    if max_x > max_y:
        height = 6.0
        width = height * aspect_ratio
    else:
        width = 6.0
        height = width * aspect_ratio

    sns.set_theme(rc={'figure.figsize': (width, height)})

    sns.scatterplot(
        data=dataset,
        x='x',
        y='y',
        size='cost',
        legend=False
    )

    if isinstance(solution, pd.DataFrame):
        x = solution['x'].to_numpy()
        y = solution['y'].to_numpy()

        for i in range(len(x) - 1):
            plt.plot([x[i], x[i + 1]], [y[i], y[i + 1]], color='b', linestyle='-')

        plt.plot([x[-1], x[0]], [y[-1], y[0]], color='b', linestyle='-')

In [ ]:
plot(dataset_A)
plt.show()

In [ ]:
plot(dataset_B)
plt.show()

# Local Search



In [ ]:
def generate_random_solution(dataset):
    size = int(len(dataset) * 0.5 + 0.5)

    return dataset.loc[np.random.choice(dataset.values.shape[0], size=size, replace=False)]

In [ ]:
def generate_greedy_cycle(dataset, distance_matrix, starting_node):
    size = int(len(dataset) * 0.5 + 0.5)
    num_nodes = len(dataset)

    distance_matrix = distance_matrix.copy()

    remaining_nodes = set(range(num_nodes))
    remaining_nodes.remove(starting_node)
    solution = [starting_node]

    nearest_node = np.argmin(distance_matrix[starting_node, list(remaining_nodes)])
    nearest_node_idx = list(remaining_nodes)[nearest_node]
    solution.append(nearest_node_idx)
    remaining_nodes.remove(nearest_node_idx)

    while len(solution) < size:
        best_insertion_cost = float('inf')
        best_insertion = None

        for node_idx in remaining_nodes:
            for i in range(len(solution)):
                next_i = (i + 1) % len(solution)

                current_cost = (
                    distance_matrix[solution[i], node_idx] +
                    distance_matrix[node_idx, solution[next_i]] -
                    distance_matrix[solution[i], solution[next_i]]
                )
                
                if current_cost < best_insertion_cost:
                    best_insertion_cost = current_cost
                    best_insertion = (node_idx, i)
        
        solution.insert(best_insertion[1] + 1, best_insertion[0])
        remaining_nodes.remove(best_insertion[0])
    
    return dataset.loc[solution]

In [ ]:
def objective_change_two_nodes(distance_matrix, solution, i, j):
    if i == j:
        return np.int64(0.0)

    n = len(solution)
    a, b = solution[i], solution[j]

    a_prev = solution[i - 1] if i > 0 else solution[-1]
    a_next = solution[(i + 1) % n]
    b_prev = solution[j - 1] if j > 0 else solution[-1]
    b_next = solution[(j + 1) % n]

    # Edges to be removed and added
    edges_removed = np.array([[a_prev, a], [a, a_next], [b_prev, b], [b, b_next]])
    edges_added = np.array([[a_prev, b], [b, a_next], [b_prev, a], [a, b_next]])

    # Calculate delta
    delta = distance_matrix[edges_removed[:, 0], edges_removed[:, 1]].sum() - \
            distance_matrix[edges_added[:, 0], edges_added[:, 1]].sum()

    return delta

def objective_change_two_edges(distance_matrix, solution, i, j):
    if i >= j or (i == 0 and j == len(solution) - 1):
        return 0.0

    n = len(solution)
    a_prev = solution[i - 1]
    a = solution[i]
    b = solution[j]
    b_next = solution[(j + 1) % n]

    # Edges before and after reversal
    cost_before = distance_matrix[a_prev, a] + distance_matrix[b, b_next]
    cost_after = distance_matrix[a_prev, b] + distance_matrix[a, b_next]

    delta = cost_after - cost_before

    return delta

def objective_change_inter_route(distance_matrix, solution, i, vacant_node, node_costs):
    prev_node = solution[i - 1] if i > 0 else solution[-1]
    next_node = solution[(i + 1) % len(solution)]
    node_in_solution = solution[i]

    edge_cost_before = distance_matrix[prev_node, node_in_solution] + distance_matrix[node_in_solution, next_node]
    edge_cost_after = distance_matrix[prev_node, vacant_node] + distance_matrix[vacant_node, next_node]
    node_cost_before = node_costs[node_in_solution]
    node_cost_after = node_costs[vacant_node]

    delta = (edge_cost_after - node_cost_after) - (edge_cost_before - node_cost_before)
    return delta

def two_nodes_exchange(solution: list, i: int, j: int) -> list:
    new_solution = solution.copy()
    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution

def two_edges_exchange(solution: list, i: int, j: int) -> list:
    if i >= j:
        return solution.copy()  # No change if i >= j

    new_solution = solution.copy()
    new_solution[i : j + 1] = new_solution[i : j + 1][::-1]
    return new_solution

def inter_route_swap(
    solution: list,
    i: int,
    vacant_node: int,
    selected_nodes: set,
    non_selected_nodes: set,
) -> tuple:
    new_solution = solution.copy()
    node_in_solution = new_solution[i]
    new_solution[i] = vacant_node

    # Update the node sets
    selected_nodes = selected_nodes.copy()
    non_selected_nodes = non_selected_nodes.copy()

    selected_nodes.remove(node_in_solution)
    selected_nodes.add(vacant_node)
    non_selected_nodes.remove(vacant_node)
    non_selected_nodes.add(node_in_solution)

    return new_solution, selected_nodes, non_selected_nodes

def get_remaining_nodes(selected_nodes: set, num_nodes: int) -> set:
    return set(range(num_nodes)) - selected_nodes

def browse_intra_solutions(distance_matrix: np.ndarray, solution: list, intra_search: str) -> tuple:
    intra_neighbors = []
    n = len(solution)
    for i in range(n):
        for j in range(i + 1, n):
            if intra_search == "node":
                delta_nodes = objective_change_two_nodes(distance_matrix, solution, i, j)
                if delta_nodes < 0:
                    intra_neighbors.append((i, j, delta_nodes, "node"))
            elif intra_search == "edge":
                delta_edges = objective_change_two_edges(distance_matrix, solution, i, j)
                if delta_edges < 0:
                    intra_neighbors.append((i, j, delta_edges, "edge"))
    return intra_neighbors

def browse_inter_solutions(
    distance_matrix: np.ndarray, solution: list, non_selected_nodes: set, costs: np.ndarray
) -> list:
    inter_neighbors = []
    solution_array = np.array(solution)
    for i in range(len(solution)):
        for vacant_node in non_selected_nodes:
            inter_delta = objective_change_inter_route(
                distance_matrix, solution_array, i, vacant_node, costs
            )
            if inter_delta < 0:
                inter_neighbors.append((i, vacant_node, inter_delta, "inter"))
    return inter_neighbors

def update_solution(
    solution: list,
    best_neighbor: tuple,
    selected_nodes: set,
    non_selected_nodes: set,
) -> tuple:
    neighbor_type = best_neighbor[-1]

    if neighbor_type == "node":
        i, j = best_neighbor[:2]
        solution = two_nodes_exchange(solution, int(i), int(j))
    elif neighbor_type == "edge":
        i, j = best_neighbor[:2]
        solution = two_edges_exchange(solution, int(i), int(j))
    elif neighbor_type == "inter":
        i, vacant_node = best_neighbor[:2]
        solution, selected_nodes, non_selected_nodes = inter_route_swap(
            solution, int(i), int(vacant_node), selected_nodes, non_selected_nodes
        )
    return solution, selected_nodes, non_selected_nodes

def local_search(dataset, distance_matrix, initial_solution, search_type='greedy', intra_move_type='two_node', debug_mode=False):   
    num_nodes = len(dataset)

    initial_cost = calculate_function_cost(dataset.loc[initial_solution])

    solution = initial_solution.copy()
    selected_nodes = set(initial_solution)
    non_selected_nodes = get_remaining_nodes(selected_nodes, num_nodes)

    while True:
        intra_neighbors = browse_intra_solutions(distance_matrix, solution, intra_move_type)
        inter_neighbors = browse_inter_solutions(distance_matrix, solution, non_selected_nodes, dataset["cost"].to_numpy())

        all_neighbors = intra_neighbors + inter_neighbors

        if not all_neighbors:
            # No improvement found
            break

        if search_type == "greedy":
            np.random.shuffle(all_neighbors)
            best_neighbor = next((n for n in all_neighbors if n[2] < 0), None)
        elif search_type == "steepest":
            best_neighbor = min(all_neighbors, key=lambda x: x[2])

        if best_neighbor is None:
            # No improving neighbor found
            break

        # Save the old solution for debugging
        old_solution = solution.copy()

        # Update solution and cost
        solution, selected_nodes, non_selected_nodes = update_solution(
            solution, best_neighbor, selected_nodes, non_selected_nodes
        )
        initial_cost += best_neighbor[2]

        if debug_mode:
            # Calculate real improvement
            real_improvement = calculate_function_cost(dataset.loc[old_solution]) - calculate_function_cost(
                dataset.loc[solution]
            )

            if real_improvement != -best_neighbor[2]:
                print(f"Promised improvement: {best_neighbor[2]}")
                print(f"Real improvement: {real_improvement}")
                print(f"Operation: {best_neighbor[-1]}")
                print("===========")

            assert calculate_function_cost(dataset.loc[solution]) < calculate_function_cost(dataset.loc[old_solution])

    return dataset.loc[solution]

In [ ]:
for distance_matrix, dataset in enumerate([dataset_A, dataset_B]):
    for use_random_start in [False, True]:    
        for search_type in ['greedy', 'steepest']:
            for intra_move_type in ['two_node', 'two_edge']:
                rating = []

                for idx in range(200):
                    if use_random_start:
                        initial_solution = list(generate_random_solution(dataset).index)
                    else:
                        initial_solution = list(generate_greedy_cycle(dataset, distance_matrix=distance_matrices[distance_matrix], starting_node=idx).index)

                    regret_solution = local_search(dataset, distance_matrices[distance_matrix], initial_solution=initial_solution, search_type=search_type, intra_move_type=intra_move_type)
                    # print(f"Solution {idx}")

                    solution = list(regret_solution.index)

                    rating.append((solution, calculate_function_cost(regret_solution)))

                best_solution = sorted(rating, key=lambda x: x[1])[0]

                print(best_solution[0])
                print(f'Objective function = {best_solution[1]}')
                print('################################\n')

                print(f"Min: {sorted(rating, key=lambda x: x[1])[0][1]}")

                average = sum([obj_function for solution, obj_function in rating])/len(rating)

                print(f'Average: {average}')
                print(f"Max: {sorted(rating, key=lambda x: x[1])[-1][1]}")

                plot(dataset, solution=dataset.loc[best_solution[0]])
                plt.title(f'Strategy: {search_type}; Intra search: {intra_move_type}; algorithm: {'random' if use_random_start else 'greedy cycle'}')
                plt.show()